In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
pip install keras-rectified-adam


In [66]:
pip install tensorflow-addons

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
import numpy as np
#from tensorflow.keras.utils import Sequence
import cv2
from keras import backend as K
import os
#from tensorflow.keras import regularizers
from keras_radam import RAdam



In [69]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

In [70]:
def iou(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)

    
def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred)
    jac = (intersection + 1.) / (union - intersection + 1.)
    return K.mean(jac)



In [71]:
def weighted_binary_crossentropy(weight0=0.25, weight1=1):
  '''
  w1 and w2 are the weights for the two classes.
  Computes weighted binary crossentropy
  Use like so:  model.compile(loss=weighted_binary_crossentropy(), optimizer="adam", metrics=["accuracy"])
  '''

  def wbce( y_true, y_pred) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=-1)


  return wbce

In [72]:
def weighted_bce(y_true, y_pred):
  weights = (y_true * 59.) + 1.
  bce = K.binary_crossentropy(y_true, y_pred)
  weighted_bce = K.mean(bce * weights)
  return weighted_bce

In [73]:
from keras.utils.data_utils import Sequence
import cv2

class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, batch_size, dim, input_img_paths):
        self.batch_size = batch_size
        self.dim = dim
        self.input_img_paths = input_img_paths
        self.shuffle = False
        self.n_channels = 1
        self.on_epoch_end()
       

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        'Generate one batch of data'
        # Generate indexes of the batch
        i = idx * self.batch_size  

        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        #batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]

        # Generate data
        X = self._generate_X(batch_input_img_paths)
        y = self._generate_y(batch_input_img_paths)
        
        return X,y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        pass

    # Normalizes values of a matrix between -1 and 1
    def myNormalization(self, data):
        max_val = np.max(data)
        min_val = np.min(data)
        
        return  2 * (data[:,:] - min_val) / (max_val - min_val) - 1


    def _load_input_image(self, image_path):
        'Load image normalized'
        img = np.load(image_path)
        img = self.myNormalization(img)
        img = cv2.resize(img.astype('float32'), (224,224), interpolation=cv2.INTER_LINEAR)
        return img

    def _load_target_image(self, image_path):
        'Load target image'
        img = np.load(image_path)
        img = cv2.resize(img.astype('float32'), (224,224), interpolation=cv2.INTER_LINEAR)
        img = np.uint8(img)
        
        #img = np.reshape(img, (300,300,1))

        return img

    def _generate_X(self, batch_input_img_paths):
        'Generates data containing batch_size images'
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        
        # Generate data
        for i, path in enumerate(batch_input_img_paths):
            # Store sample
            aux =  self._load_input_image(path)
            X[i,] = np.reshape( aux,  (224,224,1) )
            
        return X
    
    def _generate_y(self, batch_input_img_paths):
        'Generates data containing batch_size masks'
        y =  np.empty((self.batch_size, *self.dim))
        
        # Generate data
        for i, path in enumerate(batch_input_img_paths):
            # Store sample
            y[i,] = np.reshape( self._load_target_image(path.replace('seismic', 'fault').replace('input','target')), (224,224,1))
            
        return y
    

In [74]:
from numpy.random import seed
#seed(12345)
#import tensorflow as tf
#tf.random.set_seed(1234)
import os
import random
import numpy as np
import skimage
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard, EarlyStopping
from keras.layers import Input, Activation, Conv2D, MaxPooling2D, UpSampling2D, Dense, Cropping2D, Dropout, Concatenate, BatchNormalization
from keras import backend as keras
import numpy as np
#from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
from keras.models import Sequential


In [75]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [76]:
def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [77]:
def showHistory(history):
  # list all data in history
  print(history.history.keys())
  fig = plt.figure(figsize=(10,6))

  # summarize history for accuracy
  plt.plot(history.history['dice_coef'])
  plt.plot(history.history['val_dice_coef'])
  plt.title('Model accuracy',fontsize=20)
  plt.ylabel('dice_coef',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'val'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()

  # summarize history for loss
  fig = plt.figure(figsize=(10,6))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss',fontsize=20)
  plt.ylabel('Loss',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'val'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()
 


In [78]:

def unet (pretrained_weights = None,bn = False, do= False, input_size = (None,None,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1) if bn else conv1
    drop1 = Dropout(0.4)(conv1) if do else conv1
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2) if bn else conv2
    drop2 = Dropout(0.4)(conv2) if do else conv2
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3) if bn else conv3
    drop3 = Dropout(0.4)(conv3) if do else conv3
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4) if bn else conv4
    drop4 = Dropout(0.4)(conv4) if do else conv4
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.4)(conv5)if do else conv5

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    drop6 = Dropout(0.4)(merge6)if do else merge6
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    drop7 = Dropout(0.4)(merge7)if do else merge7
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    drop8 = Dropout(0.4)(merge8)if do else merge8
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    drop9 = Dropout(0.4)(merge9)if do else merge9
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(drop9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [79]:
'''root_dir = "/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/inpt_ampl-targt_3DDE"

# Creating partitions of the data after shuffeling
currentCls = '/target-2021-10-14'
src = root_dir+currentCls # Folder to copy images from input first
allFileNames = os.listdir(src)
#Split an array into multiple sub-arrays as views into ndarray.
#train 70%
#val 15%
#test 15%
import time
seed = np.random.randint(0, 100000)  
np.random.seed(seed) 
np.random.shuffle(allFileNames)
train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)*0.8), int(len(allFileNames)*0.90)])


'''

'root_dir = "/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/inpt_ampl-targt_3DDE"\n\n# Creating partitions of the data after shuffeling\ncurrentCls = \'/target-2021-10-14\'\nsrc = root_dir+currentCls # Folder to copy images from input first\nallFileNames = os.listdir(src)\n#Split an array into multiple sub-arrays as views into ndarray.\n#train 70%\n#val 15%\n#test 15%\nimport time\nseed = np.random.randint(0, 100000)  \nnp.random.seed(seed) \nnp.random.shuffle(allFileNames)\ntrain_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),\n                                                          [int(len(allFileNames)*0.8), int(len(allFileNames)*0.90)])\n\n\n'

In [80]:
root_dir = "/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/sismica/sismica-interpretada/dados-modelo"
currentCls = '/input'
src = root_dir+currentCls

train_FileNames = np.load(root_dir+'/names_withfaults-finalaug-train.npy')
np.random.shuffle(train_FileNames)
val_FileNames = np.load(root_dir+'/names_withfaults-finalaug-val.npy')


train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]

In [81]:
import tensorflow.compat.v1 as tf

def cross_entropy_balanced(y_true, y_pred):
    # Note: tf.nn.sigmoid_cross_entropy_with_logits expects y_pred is logits, 
    # Keras expects probabilities.
    # transform y_pred back to logits
    _epsilon = _to_tensor(K.epsilon(), y_pred.dtype.base_dtype)
    y_pred   = tf.clip_by_value(y_pred, _epsilon, 1 - _epsilon)
    y_pred   = tf.log(y_pred/ (1 - y_pred))

    y_true = tf.cast(y_true, tf.float32)

    count_neg = tf.reduce_sum(1. - y_true)
    count_pos = tf.reduce_sum(y_true)

    beta = count_neg / (count_neg + count_pos)

    pos_weight = beta / (1 - beta)

    cost = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, targets=y_true, pos_weight=pos_weight)

    cost = tf.reduce_mean(cost * (1 - beta))

    return tf.where(tf.equal(count_pos, 0.0), 0.0, cost)


def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
    x: An object to be converted (numpy array, list, tensors).
    dtype: The destination type.
    # Returns
    A tensor.
    """
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

In [82]:
import tensorflow_addons as tfa
def goTrain():
   
  
   #data load first
  input_img_paths_train =  train_FileNames
  input_img_paths_val = val_FileNames

  batch_size = 32
  image_size = (224,224,1)

  training_generator = DataGenerator(batch_size, image_size, input_img_paths_train)
  validation_generator = DataGenerator(batch_size, image_size, input_img_paths_val)
  
  custom_early_stopping = EarlyStopping(
    monitor='val_dice_coef', 
    patience=15, 
    min_delta=0.001, 
    mode='max'
  )

  #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
  model = Sequential()
  model = unet(bn= True, do=False, input_size=((224,224,1)))
  
  model.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-4), loss =  'binary_crossentropy',  metrics=[dice_coef, iou, tf.keras.metrics.Accuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  # Calculate the weights for each class so that we can balance the data
  
  model.summary()

  # checkpoint
  filepath="/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/check-unet-inter/fseg-dados_selecionados-dice/unet2D_exec_2022-05-11-EQE+{epoch:02d}.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='val_dice_coef', 
        verbose=1, save_best_only=True, mode='max')
  
  
  print("data prepared, ready to train!")
  #model.load_weights('/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/check-unet-inter/fseg-dados_selecionados-dice/unet2D_exec_2022-04-27-EQE+52.hdf5')
  #model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = [dice_coef])
  
  # Fit the model
  history_model = model.fit_generator(generator=training_generator,
    validation_data=validation_generator,callbacks=[checkpoint, custom_early_stopping], epochs=150)
  model.save('/content/drive/MyDrive/Pesquisa-Doc/Colab Notebooks/check-unet-inter/fseg-dados_selecionados-dice/unet2D_exec_2022-05-11-EQE-150epoc.hdf5')

  showHistory(history_model)

In [83]:
def main():
  goTrain()


In [ ]:
main()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_59 (Conv2D)             (None, 224, 224, 64  640         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_60 (Conv2D)             (None, 224, 224, 64  36928       ['conv2d_59[0][0]']              
                                )                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


data prepared, ready to train!
Epoch 1/150
 28/288 [=>............................] - ETA: 1:22:00 - loss: 0.6938 - dice_coef: 0.2139 - iou: 0.1304 - accuracy: 0.0000e+00 - precision_2: 0.1611 - recall_2: 0.0506